In [31]:
import cv2
from ultralytics import YOLO

model = YOLO('models/yolov8m-drone.pt')
model.to('cuda')

results = model.predict(source="https://www.youtube.com/watch?v=HnatsiFas5Y", show = True, verbose=False)

1/1: https://www.youtube.com/watch?v=HnatsiFas5Y... Success ✅ (1913 frames of shape 1920x1080 at 25.00 FPS)


WARNING ⚠️ inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

WARNING ⚠️ Waiting for stream 0
WARNING ⚠️ Waiting for stream 0
WARNING ⚠️ Waiting for stream 0
WARNING ⚠️ Waiting for stream 0
WARNING ⚠️ Waiting for stream 0
WARNING ⚠️ Waiting for stream 0
WARNING ⚠️ Waiting for stream 0
WARNING ⚠️ Waiting for stream 0
WARNING ⚠️ Waiting for stream 0
WARNING ⚠️ Waiting for stream 0
WARNING ⚠️ Waiting for strea

Exception in thread Thread-14:
Traceback (most recent call last):
  File "c:\Users\raaro\anaconda3\lib\threading.py", line 980, in _bootstrap_inner
    self.run()
  File "c:\Users\raaro\anaconda3\lib\threading.py", line 917, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\raaro\AppData\Roaming\Python\Python39\site-packages\ultralytics\data\loaders.py", line 121, in update
    cap.grab()  # .read() = .grab() followed by .retrieve()
cv2.error: Unknown C++ exception from OpenCV code


In [3]:
# ------- Import ---------------------------------------------------------------------------------------------------
import os
import cv2
import torch
import pyautogui
import numpy as np
from PIL import Image

import pygetwindow as gw

from pynput.mouse import Button, Controller

# ------- Functions -----------------------------------------------------------------------------------------------
def get_chrome_screenshot():
    try:
        chrome_window = [window for window in gw.getAllTitles() if 'Google Chrome' in window][0]
        window = gw.getWindowsWithTitle(chrome_window)[0]
        x, y, width, height = window.left, window.top, window.width, window.height
        screenshot = pyautogui.screenshot(region=(x, y, width, height))
        return cv2.cvtColor(np.array(screenshot), cv2.COLOR_RGB2BGR)
    except IndexError:
        print("Janela do Google Chrome não encontrada.")
        return None

# ------- RUN -----------------------------------------------------------------------------------------------------
device = torch.device("cuda")
model = torch.hub.load('ultralytics/yolov5', 'custom', path='models/best.pt', source='github') # Load YOLO model
model.to(device)
window_title  = "ChatGPT - Google Chrome" #  Window capture title
classes = ['Drone'] # Classes to detect
mouse = Controller()
follow_mouse = False


while True:

    # Frame preparation
    frame = get_chrome_screenshot()
    img = Image.fromarray(frame[...,::-1]) # Convert the frame to a format that YOLOv5 can process

    # Run inference on the frame (change size = change FPS)
    results = model(img, size=640)

    # Process the results and draw bounding boxes on the frame
    for result in results.xyxy[0]:
        x1, y1, x2, y2, conf, cls = result.tolist()
        if conf > 0.5 and classes[int(cls)] in classes:

            # Follow mouse
            if follow_mouse:
                mouse.position = ((x1+x2)/2,(y1+y2)/2)

            # Draw the bounding box
            cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0, 0, 255), 2)

            # Display the confidence score above the box
            text_conf = "{:.2f}%".format(conf * 100)
            cv2.putText(frame, text_conf, (int(x1), int(y1) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)

            # Display the bounding box coordinates below the box
            text_coords = "({}, {})".format(int((x1 + x2) / 2), int(y2))
            cv2.putText(frame, text_coords, (int(x1), int(y2) + 20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)

    # Shows the frame on the screen
    cv2.imshow('frame', frame)

    # Wait for a keyboard event to brake or space
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
    elif cv2.waitKey(1) & 0xFF == ord(' '):
        follow_mouse = not follow_mouse

# Release the resources
cv2.destroyAllWindows()


Using cache found in C:\Users\raaro/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2024-2-26 Python-3.9.13 torch-2.2.1+cu121 CUDA:0 (NVIDIA GeForce MX230, 2048MiB)

Fusing layers... 
Model summary: 157 layers, 7012822 parameters, 0 gradients, 15.8 GFLOPs
Adding AutoShape... 


In [1]:
# ------- Import --------------------------------------------------------------------------
import cv2
import torch
from PIL import Image

# ------- RUN --------------------------------------------------------------------------
model = torch.hub.load('ultralytics/yolov5', 'custom', path='models/best.pt', source='github') # Load YOLOv5 model
video_capture = cv2.VideoCapture(0) #  Camera video capture
classes = ['Drone'] # Classes to detect

while True:

    # Frame preparation
    ret, frame = video_capture.read() # Reads camera's frame
    img = Image.fromarray(frame[...,::-1]) # Convert the frame to a format that YOLOv5 can process

    # Run inference on the frame (change size = change FPS)
    results = model(img, size=640)

    # Process the results and draw bounding boxes on the frame
    for result in results.xyxy[0]:
        x1, y1, x2, y2, conf, cls = result.tolist()
        if conf > 0.5 and classes[int(cls)] in classes:

            # Draw the bounding box
            cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0, 0, 255), 2)

            # Display the confidence score above the box
            text_conf = "{:.2f}%".format(conf * 100)
            cv2.putText(frame, text_conf, (int(x1), int(y1) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)

            # Display the bounding box coordinates below the box
            text_coords = "({}, {})".format(int((x1 + x2) / 2), int(y2))
            cv2.putText(frame, text_coords, (int(x1), int(y2) + 20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)

    # Shows the frame on the screen
    cv2.imshow('frame', frame)

    # Wait for a keyboard event to brake
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the resources
video_capture.release()
cv2.destroyAllWindows()


Using cache found in C:\Users\raaro/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2024-2-26 Python-3.9.13 torch-2.2.1+cu121 CUDA:0 (NVIDIA GeForce MX230, 2048MiB)

Fusing layers... 
Model summary: 157 layers, 7012822 parameters, 0 gradients, 15.8 GFLOPs
Adding AutoShape... 
